# Multiprocessing
Tutorial by Jonas Wilfert, Tobias Thummerer

## License

In [1]:
# Copyright (c) 2021 Tobias Thummerer, Lars Mikelsons, Josef Kircher, Johannes Stoljar, Jonas Wilfert
# Licensed under the MIT license. 
# See LICENSE (https://github.com/thummeto/FMI.jl/blob/main/LICENSE) file in the project root for details.

## Motivation
This Julia Package *FMI.jl* is motivated by the use of simulation models in Julia. Here the FMI specification is implemented. FMI (*Functional Mock-up Interface*) is a free standard ([fmi-standard.org](http://fmi-standard.org/)) that defines a container and an interface to exchange dynamic models using a combination of XML files, binaries and C code zipped into a single file. The user can thus use simulation models in the form of an FMU (*Functional Mock-up Units*). Besides loading the FMU, the user can also set values for parameters and states and simulate the FMU both as co-simulation and model exchange simulation.

## Introduction to the example
This example shows how to parallelize the computation of an FMU in FMI.jl. We can compute a batch of FMU-evaluations in parallel with different initial settings.
Parallelization can be achieved using multithreading or using multiprocessing. This example shows **multiprocessing**, check `multithreading.ipynb` for multithreading.
Advantage of multithreading is a lower communication overhead as well as lower RAM usage.
However in some cases multiprocessing can be faster as the garbage collector is not shared.


The model used is a one-dimensional spring pendulum with friction. The object-orientated structure of the *SpringFrictionPendulum1D* can be seen in the following graphic.

![svg](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/pics/SpringFrictionPendulum1D.svg?raw=true)  


## Target group
The example is primarily intended for users who work in the field of simulations. The example wants to show how simple it is to use FMUs in Julia.


## Other formats
Besides, this [Jupyter Notebook](https://github.com/thummeto/FMI.jl/blob/examples/examples/multiprocessing.ipynb) there is also a [Julia file](https://github.com/thummeto/FMI.jl/blob/examples/examples/multiprocessing.jl) with the same name, which contains only the code cells and for the documentation there is a [Markdown file](https://github.com/thummeto/FMI.jl/blob/examples/examples/multiprocessing.md) corresponding to the notebook.  


## Getting started

### Installation prerequisites
|     | Description                       | Command                   | Alternative                                    |   
|:----|:----------------------------------|:--------------------------|:-----------------------------------------------|
| 1.  | Enter Package Manager via         | ]                         |                                                |
| 2.  | Install FMI via                   | add FMI                   | add " https://github.com/ThummeTo/FMI.jl "     |
| 3.  | Install FMIZoo via                | add FMIZoo                | add " https://github.com/ThummeTo/FMIZoo.jl "  |
| 4.  | Install FMICore via               | add FMICore               | add " https://github.com/ThummeTo/FMICore.jl " |
| 5.  | Install BenchmarkTools via        | add BenchmarkTools        |                                                |

## Code section



Adding your desired amount of processes:

In [2]:
using Distributed
n_procs = 2
addprocs(n_procs; exeflags=`--project=$(Base.active_project()) --threads=auto`, restrict=false)

2-element Vector{Int64}:
 2
 3

To run the example, the previously installed packages must be included. 

In [3]:
# imports
@everywhere using FMI
@everywhere using FMIZoo
@everywhere using BenchmarkTools

Checking that we workers have been correctly initialized:

In [4]:
workers()

@everywhere println("Hello World!")

# The following lines can be uncommented for more advanced informations about the subprocesses
# @everywhere println(pwd())
# @everywhere println(Base.active_project())
# @everywhere println(gethostname())
# @everywhere println(VERSION)
# @everywhere println(Threads.nthreads())

      From worker 3:	Hello World!
Hello World!
      From worker 2:	Hello World!


### Simulation setup

Next, the batch size and input values are defined.

In [5]:

# Best if batchSize is a multiple of the threads/cores
batchSize = 16

# Define an array of arrays randomly
input_values = collect(collect.(eachrow(rand(batchSize,2))))

16-element Vector{Vector{Float64}}:
 [0.9304487699389548, 0.5253014343494766]
 [0.1490190044881463, 0.6881818591761351]
 [0.30246999162118837, 0.433108058199541]
 [0.7245258048799801, 0.18395836006773258]
 [0.26655598901526734, 0.35787154622345296]
 [0.28006078347677543, 0.044998197417785324]
 [0.5782615377639864, 0.8404958327349106]
 [0.8255240799290269, 0.9169078838436682]
 [0.4021767363579263, 0.8716983211104894]
 [0.8509659898955919, 0.6102207437747618]
 [0.09751441171855446, 0.7627828126922829]
 [0.9470566958375604, 0.8387435271116479]
 [0.5120657363486965, 0.26198467617121923]
 [0.7202547404037747, 0.11394827663011053]
 [0.6394835748995205, 0.4536471812064551]
 [0.6563702227849733, 0.14442639155762516]

### Shared Module
For Distributed we need to embed the FMU into its own `module`. This prevents Distributed from trying to serialize and send the FMU over the network, as this can cause issues. This module needs to be made available on all processes using `@everywhere`.

In [6]:
@everywhere module SharedModule
    using FMIZoo
    using FMI

    t_start = 0.0
    t_step = 0.1
    t_stop = 10.0
    tspan = (t_start, t_stop)
    tData = collect(t_start:t_step:t_stop)

    model_fmu = FMIZoo.fmiLoad("SpringPendulum1D", "Dymola", "2022x")
end

┌ Info: fmi2Unzip(...): Successfully unzipped 153 files at `/tmp/fmijl_7ChpnV/SpringPendulum1D`.
└ @ FMIImport /home/runner/.julia/packages/FMIImport/1Yngw/src/FMI2_ext.jl:90
[ Info: fmi2Unzip(...): Successfully unzipped 153 files at `/tmp/fmijl_QhycR1/SpringPendulum1D`.
[ Info: fmi2Unzip(...): Successfully unzipped 153 files at `/tmp/fmijl_WqUKgX/SpringPendulum1D`.
┌ Info: fmi2Load(...): FMU resources location is `file:////tmp/fmijl_7ChpnV/SpringPendulum1D/resources`
└ @ FMIImport /home/runner/.julia/packages/FMIImport/1Yngw/src/FMI2_ext.jl:221
[ Info: fmi2Load(...): FMU resources location is `file:////tmp/fmijl_QhycR1/SpringPendulum1D/resources`
[ Info: fmi2Load(...): FMU supports both CS and ME, using CS as default if nothing specified.
┌ Info: fmi2Load(...): FMU supports both CS and ME, using CS as default if nothing specified.
└ @ FMIImport /home/runner/.julia/packages/FMIImport/1Yngw/src/FMI2_ext.jl:224
[ Info: fmi2Load(...): FMU resources location is `file:////tmp/fmijl_WqUKgX/S

We define a helper function to calculate the FMU and combine it into an Matrix.

In [7]:
@everywhere function runCalcFormatted(fmu, x0, recordValues=["mass.s", "mass.v"])
    data = fmiSimulateME(fmu, SharedModule.t_start, SharedModule.t_stop; recordValues=recordValues, saveat=SharedModule.tData, x0=x0, showProgress=false, dtmax=1e-4)
    return reduce(hcat, data.states.u)
end

Running a single evaluation is pretty quick, therefore the speed can be better tested with BenchmarkTools.

In [8]:
@benchmark data = runCalcFormatted(SharedModule.model_fmu, rand(2))

BenchmarkTools.Trial: 14 samples with 1 evaluation.
 Range (min … max):  358.880 ms … 382.379 ms  ┊ GC (min … max): 1.84% … 1.73%
 Time  (median):     369.689 ms               ┊ GC (median):    3.56%
 Time  (mean ± σ):   370.461 ms ±   4.860 ms  ┊ GC (mean ± σ):  3.19% ± 0.77%

                             █▃      ▃                           
  ▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▇▁██▁▇▁▁▇▁█▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  359 ms           Histogram: frequency by time          382 ms <

 Memory estimate: 128.48 MiB, allocs estimate: 1802004.

### Single Threaded Batch Execution
To compute a batch we can collect multiple evaluations. In a single threaded context we can use the same FMU for every call.

In [9]:
println("Single Threaded")
@benchmark collect(runCalcFormatted(SharedModule.model_fmu, i) for i in input_values)

Single Threaded


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 5.917 s (3.24% GC) to evaluate,
 with a memory estimate of 2.01 GiB, over 28832052 allocations.

### Multithreaded Batch Execution
In a multithreaded context we have to provide each thread it's own fmu, as they are not thread safe.
To spread the execution of a function to multiple processes, the function `pmap` can be used.

In [10]:
println("Multi Threaded")
@benchmark pmap(i -> runCalcFormatted(SharedModule.model_fmu, i), input_values)

Multi Threaded


BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  3.524 s …  3.533 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.529 s             ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.529 s ± 6.049 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.52 s        Histogram: frequency by time        3.53 s <

 Memory estimate: 81.14 KiB, allocs estimate: 1195.

As you can see, there is a significant speed-up in the median execution time. But: The speed-up is often much smaller than `n_procs` (or the number of physical cores of your CPU), this has different reasons. For a rule of thumb, the speed-up should be around `n/2` on a `n`-core-processor with `n` Julia processes.

### Unload FMU

After calculating the data, the FMU is unloaded and all unpacked data on disc is removed.

In [11]:
@everywhere fmiUnload(SharedModule.model_fmu)

### Summary

In this tutorial it is shown how multi processing with `Distributed.jl` can be used to improve the performance for calculating a Batch of FMUs.